In [2]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
# from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
# from keras.utils import np_utils
# from skimage.transform import resize   # for resizing images
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import random as rnd
from glob import glob
from tqdm import tqdm

datadir = "/home/ephy/Projects/tda_convolve_video/data/"

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
device='cuda:0'

from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.integrate import tplquad
from scipy.ndimage import rotate as scipy_rotate
from scipy.ndimage import shift as scipy_shift
from joblib import Parallel, delayed

# Import topnet utilities
import sys
sys.path.insert(0, '/home/ephy/Projects/tda_convolution/src/')
import topnet
import importlib
importlib.reload(topnet)

<module 'topnet' from '/home/ephy/Projects/tda_convolution/src/topnet.py'>

# Data prep

In [2]:
f = open(datadir+"original/ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]

# open the .txt file which have names of test videos
f = open(datadir+"original/ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]

In [3]:
# creating tags
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

allVids = train.append(test, ignore_index=True)

In [4]:
# Grabbing at ~ 25fps we can get 

In [5]:
# storing the frames from training videos
for i in tqdm(range(allVids.shape[0])):
    count = 0
    videoFile = allVids['video_name'][i]
    cap = cv2.VideoCapture(datadir + 'original/UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = 1 #frame sample rate
    x=1
    i=0
    while(cap.isOpened() and i<13):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename =datadir+'altered/allUCF101_FFR/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
            i+=1
    cap.release()

100%|██████████| 13320/13320 [11:08<00:00, 19.93it/s]


In [6]:
# getting the names of all the images
images = glob(datadir + "altered/allUCF101_FFR/*.jpg")

all_image = []
all_vid = []
all_class = []
all_frame = []
for i in tqdm(range(len(images))):
    n = images[i].split('/')[8]
    all_image.append(n)
    all_vid.append('_'.join(n.split('_')[1:4]))
    all_class.append(n.split('_')[1])
    all_frame.append(n.split('_')[4].split('.')[0].split('frame')[1])
# # storing the images and their class in a dataframe
all_data = pd.DataFrame()
all_data['image'] = all_image
all_data['class'] = all_class
all_data['clip'] = all_vid
all_data['frame'] = all_frame

# # converting the dataframe into csv file 
all_data.to_csv(datadir + 'altered/all_data_guide_FFR.csv',header=True, index=False)

100%|██████████| 173160/173160 [00:00<00:00, 449948.20it/s]


# Create 3-fold batches

In [3]:
allVids = pd.read_csv(datadir + 'altered/all_data_guide_FFR.csv')
allVids = allVids.sort_values(by=['clip', 'frame'])
allVids_dedup = allVids.drop_duplicates(subset=['clip'], keep='first').copy()

In [4]:
# Make an interger based class to make tracking tests more simple
classXwalk = pd.DataFrame({'class': np.unique(allVids_dedup['class'])})
classXwalk['ind'] = np.array(list(range(0,101)))
allVids_dedup['class_int'] = [int(classXwalk['ind'][classXwalk['class']==c]) for c in allVids_dedup['class']]

In [5]:
# # Go through and load videos as 3d volume arrays
# clips = np.unique(allVids['clip'])

# all_image = []
# for i in tqdm(range(len(clips))):
#     stack = []
#     for imageName in allVids[allVids['clip']==clips[i]]['image']:
#         image = cv2.imread(datadir + 'altered/allUCF101_FFR/'+imageName)
#         image = Image.fromarray(image , 'RGB')
#         image = image.resize((100 , 100))
#         image = np.array(image)/255
#         image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
#         stack.append(image)
#     all_image.append(np.array(stack))

# all_image = np.array(all_image)

In [6]:
# np.save('/home/ephy/Projects/tda_convolve_video/data/altered/allVidStack100x100_FFR_13frames.npy',all_image)
all_image = np.load('/home/ephy/Projects/tda_convolve_video/data/altered/allVidStack100x100_FFR_13frames.npy', allow_pickle=True)

In [7]:
skf = StratifiedKFold(n_splits=3, random_state=2)
print(skf)

StratifiedKFold(n_splits=3, random_state=2, shuffle=False)


# Load 3x5x5 filters

In [35]:
# grids = topnet.legendre_klein_bottle(8,8,7,None)
# np.save('/home/ephy/Projects/tda_convolve_video/src/python3/KleinFeatures_55.npy', grids)
filters0 = np.load('/home/ephy/Projects/tda_convolve_video/src/python3//KleinFeatures_55.npy', allow_pickle=True)

In [36]:
filters0 = filters0.reshape(-1,1,7,7)

(64, 1, 7, 7)

In [8]:
# filters = np.load('/home/ephy/Projects/tda_convolve_video/src/python3/VideoFeatures_355.npy', allow_pickle=True)
# With rotations
filters = np.load('/home/ephy/Projects/tda_convolve_video/src/python3/VideoFeatures_355_movandrot.npy', allow_pickle=True)
flat_filters = filters.reshape(-1, *filters.shape[-3:])

# Implement Network

In [97]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv0 = nn.Conv3d(1, 64, (1,7,7),stride=(1,2,2))
        
        self.conv1 = nn.Conv3d(64, 180, (3,5,5))
        self.conv2 = nn.Conv3d(180, 180, (3,5,5))
        self.conv3 = nn.Conv3d(180, 180, (3,3,3))
        self.conv4 = nn.Conv3d(180, 36, (1,3,3))
        
        x= torch.randn(1,1,9,112,112)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 750)
        self.fc2 = nn.Linear(750, 200)
        self.fc3 = nn.Linear(200, 101)
        
        self.batch1 = nn.BatchNorm1d(self._to_linear)
        self.batch2 = nn.BatchNorm1d(750)
        self.batch3 = nn.BatchNorm1d(200)
#         self.drop1 = nn.Dropout(p=0.2)
        self.drop2 = nn.Dropout(p=0.2)
        
        
    def convs(self, x):
        x = F.max_pool3d(F.relu(self.conv0(x)),(1,3,3),stride=(1,2,2))
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]*x[0].shape[3]
            
        return x
    
    def forward(self, x):
        # reshape inside of array
        x = self.convs(x)
#         x = self.drop1(x)
        x = x.view(-1, self._to_linear)
#         x = self.batch1(x)
        x = F.leaky_relu(self.fc1(x))
#         x = self.batch2(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.batch3(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [98]:
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
torch.cuda.memory_allocated()*1e-9

0.08227020800000001

# Define Test

In [99]:
def test(net, X, y, batchSize):
    confusionMatrix = np.zeros([101,101], dtype=np.int8)
    testingSeq = list(range(0,y.shape[0]+1,batchSize))
    testingSeq.append(y.shape[0]+1)
    testingSeq = np.array(testingSeq)
    with torch.no_grad():
        for i in range(len(testingSeq)-1):
            data = X[testingSeq[i]:testingSeq[i+1]].view(-1,1,9,100,100).to(device)
            out = torch.argmax(net(data),axis=1).cpu()
            yt = torch.argmax(y[testingSeq[i]:testingSeq[i+1]],axis=1)
            for pred,label in zip(out,yt):
                confusionMatrix[label][pred] +=1
        del data
        del out        
        return confusionMatrix

In [18]:
EPOCHS = 40
# Batch size of 26+1, because normalization requires same batch size and 4472%26=0 (very close to even split) (same logic for training)
testingBatchSize=20
trainingBatchSize=10
results = [[],[],[]]
lossrec = [[],[],[]]

rnd.seed(1)
classMat = np.eye(101,dtype=np.float)
for foldIndex in range(3):
# for foldIndex, (train_index, test_index) in enumerate(skf.split(all_image, allVids_dedup['class'])):
    net = Net().to(device)
    optimizer = optim.Adam(net.parameters(), lr=1e-5)
    # Set filters in 2 layers
    with torch.no_grad():
        for i,weights in enumerate(flat_filters):
            net.conv1.weight[i][0] = torch.nn.Parameter(torch.tensor(weights)).to(device)
#             for g,filts in enumerate(net.conv2.weight):
#                 net.conv2.weight[i][g] = torch.nn.Parameter(torch.tensor(flat_filters[g])).to(device)
        net.conv1.requires_grad=False
#         net.conv2.requires_grad=False
    print('Network Reset.')

#     ## Check our work
#     with torch.no_grad():
#         for i,weights in enumerate(flat_filters):
#             print(net.conv2.weight.shape)
#             plt.imshow(net.conv2.weight[i][9][0].cpu(), cmap='gray')
#             plt.show
#             break        
                
    #Shuffle does not actually work, so just manually shuffle...
#     rnd.shuffle(train_index)
#     rnd.shuffle(test_index)
#     y_train = allVids_dedup.iloc[train_index]['class_int']
#     y_test = allVids_dedup.iloc[test_index]['class_int']
#     X_train = all_image[train_index]
#     X_test = all_image[test_index]

    X_train, X_test, y_train, y_test = train_test_split(np.stack(all_image), allVids_dedup['class_int'],
                                                        shuffle=True, random_state=foldIndex,
                                                        test_size=0.33, stratify = allVids_dedup['class_int'])


    y_train = torch.tensor([classMat[c] for c in y_train],dtype=torch.float32)
    y_test = torch.tensor([classMat[c] for c in y_test],dtype=torch.float32)
    X_train = torch.tensor(X_train,dtype=torch.float32)
    X_test =  torch.tensor(X_test,dtype=torch.float32)
    y_train = torch.tensor(np.array(y_train),dtype=torch.float32)
    y_test = torch.tensor(np.array(y_test),dtype=torch.float32)

    trainingSeq = list(range(0,y_train.shape[0],trainingBatchSize))
    trainingSeq.append(y_train.shape[0]+1)
    trainingSeq = np.array(trainingSeq)

    # Grab 5 tests in final epoch (4 + last one run after training loop)
    finalEpochTests = list(range(int((len(trainingSeq)-1)/5),len(trainingSeq)-1,int((len(trainingSeq)-1)/5)))

    for epoch in range(EPOCHS):
        for i in range(len(trainingSeq)-1):
            X=X_train[trainingSeq[i]:trainingSeq[i+1]].view(-1,1,7,100,100)
            X=X.to(device)
            y=y_train[trainingSeq[i]:trainingSeq[i+1]]
            y=y.to(device)
            net.zero_grad()
            output = net(X)
            loss = nn.CrossEntropyLoss()(output, torch.argmax(y, axis=1))
            lossrec[foldIndex].append(float(loss.tolist()))
            loss.backward()
            optimizer.step()
            del loss
            del output
            del X
            del y
            if epoch==(EPOCHS-1) and (i in finalEpochTests):
                print(foldIndex,'Final Epoch',i)
                results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
        print('Fold: ', foldIndex, 'Epoch: ',epoch)
        results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
        cm = results[foldIndex][len(results[foldIndex])-1]
        print('Accuracy: ' , np.sum(np.diag(cm))/np.sum(cm))

    np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/losses_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',lossrec)
    np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/confuseMats_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',results)    

Network Reset.
Fold:  0 Epoch:  0
Accuracy:  0.0905368516833485
Fold:  0 Epoch:  1
Accuracy:  0.15764331210191082
Fold:  0 Epoch:  2
Accuracy:  0.24522292993630573
Fold:  0 Epoch:  3
Accuracy:  0.33371246587807096
Fold:  0 Epoch:  4
Accuracy:  0.3862602365787079
Fold:  0 Epoch:  5
Accuracy:  0.4442675159235669
Fold:  0 Epoch:  6
Accuracy:  0.4752047315741583
Fold:  0 Epoch:  7
Accuracy:  0.4986351228389445
Fold:  0 Epoch:  8
Accuracy:  0.5161510464058234
Fold:  0 Epoch:  9
Accuracy:  0.5238853503184714
Fold:  0 Epoch:  10
Accuracy:  0.531164695177434
Fold:  0 Epoch:  11
Accuracy:  0.5302547770700637
Fold:  0 Epoch:  12
Accuracy:  0.5400363967242948
Fold:  0 Epoch:  13
Accuracy:  0.5450409463148317
Fold:  0 Epoch:  14
Accuracy:  0.5402638762511374
Fold:  0 Epoch:  15
Accuracy:  0.54049135577798
Fold:  0 Epoch:  16
Accuracy:  0.5475432211101001
Fold:  0 Epoch:  17
Accuracy:  0.543221110100091
Fold:  0 Epoch:  18
Accuracy:  0.551410373066424
Fold:  0 Epoch:  19
Accuracy:  0.54322111010009

KeyboardInterrupt: 

In [13]:
# Test opening first layer mid-train (works)
# Test more frames
all_image=all_image[:,0:9,:,:]
all_image.shape

(13320, 9, 100, 100)

In [13]:
EPOCHS = 40
# Batch size of 26+1, because normalization requires same batch size and 4472%26=0 (very close to even split) (same logic for training)
testingBatchSize=20
trainingBatchSize=10
results = [[],[],[]]
lossrec = [[],[],[]]

rnd.seed(1)
classMat = np.eye(101,dtype=np.float)
for foldIndex in range(3):
# for foldIndex, (train_index, test_index) in enumerate(skf.split(all_image, allVids_dedup['class'])):
    net = Net().to(device)
    optimizer = optim.Adam(net.parameters(), lr=1e-5)
    # Set filters in 2 layers
    with torch.no_grad():
        for i,weights in enumerate(flat_filters):
            net.conv1.weight[i][0] = torch.nn.Parameter(torch.tensor(weights)).to(device)
#             for g,filts in enumerate(net.conv2.weight):
#                 net.conv2.weight[i][g] = torch.nn.Parameter(torch.tensor(flat_filters[g])).to(device)
        net.conv1.requires_grad=False
#         net.conv2.requires_grad=False
    print('Network Reset.')

#     ## Check our work
#     with torch.no_grad():
#         for i,weights in enumerate(flat_filters):
#             print(net.conv2.weight.shape)
#             plt.imshow(net.conv2.weight[i][9][0].cpu(), cmap='gray')
#             plt.show
#             break        
                
    #Shuffle does not actually work, so just manually shuffle...
#     rnd.shuffle(train_index)
#     rnd.shuffle(test_index)
#     y_train = allVids_dedup.iloc[train_index]['class_int']
#     y_test = allVids_dedup.iloc[test_index]['class_int']
#     X_train = all_image[train_index]
#     X_test = all_image[test_index]

    X_train, X_test, y_train, y_test = train_test_split(np.stack(all_image), allVids_dedup['class_int'],
                                                        shuffle=True, random_state=foldIndex,
                                                        test_size=0.33, stratify = allVids_dedup['class_int'])


    y_train = torch.tensor([classMat[c] for c in y_train],dtype=torch.float32)
    y_test = torch.tensor([classMat[c] for c in y_test],dtype=torch.float32)
    X_train = torch.tensor(X_train,dtype=torch.float32)
    X_test =  torch.tensor(X_test,dtype=torch.float32)
    y_train = torch.tensor(np.array(y_train),dtype=torch.float32)
    y_test = torch.tensor(np.array(y_test),dtype=torch.float32)

    trainingSeq = list(range(0,y_train.shape[0],trainingBatchSize))
    trainingSeq.append(y_train.shape[0]+1)
    trainingSeq = np.array(trainingSeq)

    # Grab 5 tests in final epoch (4 + last one run after training loop)
    finalEpochTests = list(range(int((len(trainingSeq)-1)/5),len(trainingSeq)-1,int((len(trainingSeq)-1)/5)))

    for epoch in range(EPOCHS):
        for i in range(len(trainingSeq)-1):
            X=X_train[trainingSeq[i]:trainingSeq[i+1]].view(-1,1,9,100,100)
            X=X.to(device)
            y=y_train[trainingSeq[i]:trainingSeq[i+1]]
            y=y.to(device)
            net.zero_grad()
            output = net(X)
            loss = nn.CrossEntropyLoss()(output, torch.argmax(y, axis=1))
            lossrec[foldIndex].append(float(loss.tolist()))
            loss.backward()
            optimizer.step()
            del loss
            del output
            del X
            del y
            if epoch==(EPOCHS-1) and (i in finalEpochTests):
                print(foldIndex,'Final Epoch',i)
                results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
        print('Fold: ', foldIndex, 'Epoch: ',epoch)
        results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
        cm = results[foldIndex][len(results[foldIndex])-1]
        print('Accuracy: ' , np.sum(np.diag(cm))/np.sum(cm))

    np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/losses_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',lossrec)
    np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/confuseMats_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',results)    

Fold:  0 Epoch:  0
Accuracy:  0.5657415832575068
Fold:  0 Epoch:  1
Accuracy:  0.5666515013648772
Fold:  0 Epoch:  2
Accuracy:  0.564604185623294
Fold:  0 Epoch:  3
Accuracy:  0.5648316651501365
Fold:  0 Epoch:  4
Accuracy:  0.565059144676979
Fold:  0 Epoch:  5
Accuracy:  0.5675614194722475
Fold:  0 Epoch:  6
Accuracy:  0.5686988171064604
Fold:  0 Epoch:  7
Accuracy:  0.5625568698817106
Fold:  0 Epoch:  8
Accuracy:  0.5630118289353958
Fold:  0 Epoch:  9
Accuracy:  0.5609645131938126
Fold:  0 Epoch:  10
Accuracy:  0.5614194722474978
Fold:  0 Epoch:  11
Accuracy:  0.5727934485896269
Fold:  0 Epoch:  12
Accuracy:  0.5621019108280255
Fold:  0 Epoch:  13
Accuracy:  0.5673339399454049
Fold:  0 Epoch:  14
Accuracy:  0.5659690627843494
Fold:  0 Epoch:  15
Accuracy:  0.5605095541401274
Fold:  0 Epoch:  16
Accuracy:  0.5550500454959054
Fold:  0 Epoch:  17


KeyboardInterrupt: 

In [19]:
EPOCHS = 40
# Batch size of 26+1, because normalization requires same batch size and 4472%26=0 (very close to even split) (same logic for training)
testingBatchSize=20
trainingBatchSize=20
results = [[],[],[]]
lossrec = [[],[],[]]

rnd.seed(1)
classMat = np.eye(101,dtype=np.float)
for foldIndex in range(3):
# for foldIndex, (train_index, test_index) in enumerate(skf.split(all_image, allVids_dedup['class'])):
    net = Net().to(device)
    optimizer = optim.Adam(net.parameters(), lr=1e-5)
    # Set filters in 2 layers
    with torch.no_grad():
        for i,weights in enumerate(flat_filters):
            net.conv1.weight[i][0] = torch.nn.Parameter(torch.tensor(weights)).to(device)
#             for g,filts in enumerate(net.conv2.weight):
#                 net.conv2.weight[i][g] = torch.nn.Parameter(torch.tensor(flat_filters[g])).to(device)
        net.conv1.requires_grad=False
#         net.conv2.requires_grad=False
    print('Network Reset.')

#     ## Check our work
#     with torch.no_grad():
#         for i,weights in enumerate(flat_filters):
#             print(net.conv2.weight.shape)
#             plt.imshow(net.conv2.weight[i][9][0].cpu(), cmap='gray')
#             plt.show
#             break        
                
    #Shuffle does not actually work, so just manually shuffle...
#     rnd.shuffle(train_index)
#     rnd.shuffle(test_index)
#     y_train = allVids_dedup.iloc[train_index]['class_int']
#     y_test = allVids_dedup.iloc[test_index]['class_int']
#     X_train = all_image[train_index]
#     X_test = all_image[test_index]

    X_train, X_test, y_train, y_test = train_test_split(np.stack(all_image), allVids_dedup['class_int'],
                                                        shuffle=True, random_state=foldIndex,
                                                        test_size=0.33, stratify = allVids_dedup['class_int'])


    y_train = torch.tensor([classMat[c] for c in y_train],dtype=torch.float32)
    y_test = torch.tensor([classMat[c] for c in y_test],dtype=torch.float32)
    X_train = torch.tensor(X_train,dtype=torch.float32)
    X_test =  torch.tensor(X_test,dtype=torch.float32)
    y_train = torch.tensor(np.array(y_train),dtype=torch.float32)
    y_test = torch.tensor(np.array(y_test),dtype=torch.float32)

    trainingSeq = list(range(0,y_train.shape[0],trainingBatchSize))
    trainingSeq.append(y_train.shape[0]+1)
    trainingSeq = np.array(trainingSeq)

    # Grab 5 tests in final epoch (4 + last one run after training loop)
    finalEpochTests = list(range(int((len(trainingSeq)-1)/5),len(trainingSeq)-1,int((len(trainingSeq)-1)/5)))

    for epoch in range(EPOCHS):
        for i in range(len(trainingSeq)-1):
            X=X_train[trainingSeq[i]:trainingSeq[i+1]].view(-1,1,7,100,100)
            X=X.to(device)
            y=y_train[trainingSeq[i]:trainingSeq[i+1]]
            y=y.to(device)
            net.zero_grad()
            output = net(X)
            loss = nn.CrossEntropyLoss()(output, torch.argmax(y, axis=1))
            lossrec[foldIndex].append(float(loss.tolist()))
            loss.backward()
            optimizer.step()
            del loss
            del output
            del X
            del y
            if epoch==(EPOCHS-1) and (i in finalEpochTests):
                print(foldIndex,'Final Epoch',i)
                results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
        print('Fold: ', foldIndex, 'Epoch: ',epoch)
        results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
        cm = results[foldIndex][len(results[foldIndex])-1]
        print('Accuracy: ' , np.sum(np.diag(cm))/np.sum(cm))

    np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/losses_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',lossrec)
    np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/confuseMats_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',results)    

Fold:  0 Epoch:  0
Accuracy:  0.17333939945404914
Fold:  0 Epoch:  1


KeyboardInterrupt: 

In [18]:
results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
cm = results[foldIndex][len(results[foldIndex])-1]
print('Accuracy: ' , np.sum(np.diag(cm))/np.sum(cm))

Accuracy:  0.09440400363967243


In [19]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convRescale = nn.Conv3d(1, 64, (1,7,7))
        
        
        self.conv1 = nn.Conv3d(64, 180, (3,5,5))
        self.conv2 = nn.Conv3d(180, 180, (3,5,5))
        self.conv3 = nn.Conv3d(180, 180, (3,5,5))
        self.conv4 = nn.Conv3d(180, 180, (3,5,5))
        self.conv5 = nn.Conv3d(180, 36, (1,5,5))
        
        self.batchConv1 = nn.BatchNorm3d(180)
        self.batchConv2 = nn.BatchNorm3d(180)
        self.batchConv3 = nn.BatchNorm3d(36)
        
        self.down1 = nn.Upsample((5,92,92),mode='trilinear',align_corners=False)
        self.down2 = nn.Upsample((1,84,84),mode='trilinear',align_corners=False)

        x= torch.randn(1,1,9,100,100)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 700)
        self.fc2 = nn.Linear(700, 200)
        self.fc3 = nn.Linear(200, 101)        
        
        self.batch1 = nn.BatchNorm1d(self._to_linear)
        self.batch2 = nn.BatchNorm1d(750)
        self.batch3 = nn.BatchNorm1d(200)
        self.drop2 = nn.Dropout(p=0.2)
        
        
    def convs(self, x):
        x = F.max_pool3d(self.convRescale(x),(1,3,3))
        print(x.shape)
        inp1 = self.down1(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.batchConv1(x)
        inp2 = self.down2(x)
        x+=inp1
        del inp1
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.batchConv2(x)
        x+=inp2
        del inp2
        x = F.relu(self.conv5(x))
        x = self.batchConv3(x)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]*x[0].shape[3]
            
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.batch3(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [20]:
net = ResNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
torch.cuda.memory_allocated()*1e-9

torch.Size([1, 64, 9, 31, 31])


RuntimeError: The size of tensor a (23) must match the size of tensor b (92) at non-singleton dimension 4

In [11]:
all_image = all_image[:,0:9,...]

In [12]:
EPOCHS = 40
# Batch size of 26+1, because normalization requires same batch size and 4472%26=0 (very close to even split) (same logic for training)
testingBatchSize=10
trainingBatchSize=10
results = [[],[],[]]
lossrec = [[],[],[]]
foldIndex=0
rnd.seed(1)
classMat = np.eye(101,dtype=np.float)
# for foldIndex in range(3):
# for foldIndex, (train_index, test_index) in enumerate(skf.split(all_image, allVids_dedup['class'])):
net = ResNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
# Set filters in 2 layers
with torch.no_grad():
    for i,weights in enumerate(flat_filters):
        net.conv1.weight[i][0] = torch.nn.Parameter(torch.tensor(weights)).to(device)
#             for g,filts in enumerate(net.conv2.weight):
#                 net.conv2.weight[i][g] = torch.nn.Parameter(torch.tensor(flat_filters[g])).to(device)
    net.conv1.requires_grad=False
#         net.conv2.requires_grad=False
print('Network Reset.')

#     ## Check our work
#     with torch.no_grad():
#         for i,weights in enumerate(flat_filters):
#             print(net.conv2.weight.shape)
#             plt.imshow(net.conv2.weight[i][9][0].cpu(), cmap='gray')
#             plt.show
#             break        

#Shuffle does not actually work, so just manually shuffle...
#     rnd.shuffle(train_index)
#     rnd.shuffle(test_index)
#     y_train = allVids_dedup.iloc[train_index]['class_int']
#     y_test = allVids_dedup.iloc[test_index]['class_int']
#     X_train = all_image[train_index]
#     X_test = all_image[test_index]

X_train, X_test, y_train, y_test = train_test_split(np.stack(all_image), allVids_dedup['class_int'],
                                                    shuffle=True, random_state=foldIndex,
                                                    test_size=0.33, stratify = allVids_dedup['class_int'])


y_train = torch.tensor([classMat[c] for c in y_train],dtype=torch.float32)
y_test = torch.tensor([classMat[c] for c in y_test],dtype=torch.float32)
X_train = torch.tensor(X_train,dtype=torch.float32)
X_test =  torch.tensor(X_test,dtype=torch.float32)
y_train = torch.tensor(np.array(y_train),dtype=torch.float32)
y_test = torch.tensor(np.array(y_test),dtype=torch.float32)

trainingSeq = list(range(0,y_train.shape[0],trainingBatchSize))
trainingSeq.append(y_train.shape[0]+1)
trainingSeq = np.array(trainingSeq)

# Grab 5 tests in final epoch (4 + last one run after training loop)
finalEpochTests = list(range(int((len(trainingSeq)-1)/5),len(trainingSeq)-1,int((len(trainingSeq)-1)/5)))

for epoch in range(EPOCHS):
    for i in range(len(trainingSeq)-1):
        X=X_train[trainingSeq[i]:trainingSeq[i+1]].view(-1,1,9,100,100)
        X=X.to(device)
        y=y_train[trainingSeq[i]:trainingSeq[i+1]]
        y=y.to(device)
        net.zero_grad()
        output = net(X)
        loss = nn.CrossEntropyLoss()(output, torch.argmax(y, axis=1))
        lossrec[foldIndex].append(float(loss.tolist()))
        loss.backward()
        optimizer.step()
        del loss
        del output
        del X
        del y
        if epoch==(EPOCHS-1) and (i in finalEpochTests):
            print(foldIndex,'Final Epoch',i)
            results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
    print('Fold: ', foldIndex, 'Epoch: ',epoch)
    results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
    cm = results[foldIndex][len(results[foldIndex])-1]
    print('Accuracy: ' , np.sum(np.diag(cm))/np.sum(cm))

np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/losses_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',lossrec)
np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/confuseMats_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',results)    

Network Reset.
Fold:  0 Epoch:  0
Accuracy:  0.13034576888080074
Fold:  0 Epoch:  1
Accuracy:  0.23589626933575977
Fold:  0 Epoch:  2
Accuracy:  0.3237033666969973
Fold:  0 Epoch:  3
Accuracy:  0.38967242948134667
Fold:  0 Epoch:  4
Accuracy:  0.4356232939035487
Fold:  0 Epoch:  5
Accuracy:  0.48316651501364877
Fold:  0 Epoch:  6
Accuracy:  0.5095541401273885
Fold:  0 Epoch:  7
Accuracy:  0.5336669699727025
Fold:  0 Epoch:  8
Accuracy:  0.5452684258416742
Fold:  0 Epoch:  9
Accuracy:  0.5534576888080073
Fold:  0 Epoch:  10
Accuracy:  0.5464058234758872
Fold:  0 Epoch:  11
Accuracy:  0.5570973612374887
Fold:  0 Epoch:  12
Accuracy:  0.564604185623294
Fold:  0 Epoch:  13
Accuracy:  0.5648316651501365
Fold:  0 Epoch:  14
Accuracy:  0.5625568698817106
Fold:  0 Epoch:  15
Accuracy:  0.5666515013648772
Fold:  0 Epoch:  16
Accuracy:  0.5639217470427661
Fold:  0 Epoch:  17
Accuracy:  0.5609645131938126
Fold:  0 Epoch:  18
Accuracy:  0.5555050045495905
Fold:  0 Epoch:  19
Accuracy:  0.546633303

KeyboardInterrupt: 

In [ ]:
# ADD BATCHNORM TO ALL LAYERS

In [3]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convRescale = nn.Conv3d(1, 180, (1,7,7))
        
        
        self.conv1 = nn.Conv3d(180, 180, (3,5,5))
        self.conv2 = nn.Conv3d(180, 180, (3,5,5))
        self.conv3 = nn.Conv3d(180, 180, (3,5,5))
        self.conv4 = nn.Conv3d(180, 180, (3,5,5))
        self.conv5 = nn.Conv3d(180, 75, (3,5,5))
        
        self.batchConv1 = nn.BatchNorm3d(180)
        self.batchConv2 = nn.BatchNorm3d(180)
        self.batchConv3 = nn.BatchNorm3d(75)
        
        self.up = nn.Upsample((9,31,31),mode='trilinear',align_corners=False)
        
        x= torch.randn(1,1,9,100,100)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 700)
        self.fc2 = nn.Linear(700, 200)
        self.fc3 = nn.Linear(200, 101)        
        
        self.batch1 = nn.BatchNorm1d(self._to_linear)
        self.batch2 = nn.BatchNorm1d(750)
        self.batch3 = nn.BatchNorm1d(200)
        self.drop2 = nn.Dropout(p=0.2)
        
        
    def convs(self, x):
        x = F.max_pool3d(self.convRescale(x),(1,3,3))
        inp1 = x
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.batchConv1(x)
        x = self.up(x)
        inp2 = x
        x+=inp1
        del inp1
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.batchConv2(x)
        x = self.up(x)
        x+=inp2
        del inp2
        x = F.relu(self.conv5(x))
        x = self.batchConv3(x)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]*x[0].shape[3]
            
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.batch3(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [4]:
net = ResNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
torch.cuda.memory_allocated()*1e-9

1.1214182400000001